In [290]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [291]:
# Load the dataset
df = pd.read_csv('database.csv', header=0)


In [292]:
X = df[[ 'h', 't', 'l', 'fy', 'fc', 'A']]
y = df['n']

In [293]:
# Scale features and target variable to [0, 1]
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
y = scaler.fit_transform(y.values.reshape(-1, 1))

In [294]:
y = y * 2 - 1
X = X * 2 - 1

In [295]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [296]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(40, activation='tanh'),
    # tf.keras.layers.Dense(20, activation='tanh'),
    tf.keras.layers.Dense(40, activation='tanh'),
    tf.keras.layers.Dense(1,activation='tanh')  # Output layer for regression
])


In [297]:
# Compile the model
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=["mean_absolute_percentage_error"])

In [298]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_mean_absolute_percentage_error', patience=500, restore_best_weights=True)
# Model checkpoint callback
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True)
# TensorBoard callback for profiling
tensorboard = TensorBoard(log_dir="logs/")


In [299]:
# Train the model
model.fit(X_train, y_train, epochs=500000, batch_size=5000, verbose=2, validation_split=0.1, callbacks=[early_stopping, checkpoint, tensorboard])


Epoch 1/500000
1/1 - 1s - loss: 0.3588 - mean_absolute_percentage_error: 109.5781 - val_loss: 0.2826 - val_mean_absolute_percentage_error: 104.7452 - 558ms/epoch - 558ms/step
Epoch 2/500000
1/1 - 0s - loss: 0.3360 - mean_absolute_percentage_error: 105.3352 - val_loss: 0.2706 - val_mean_absolute_percentage_error: 101.3626 - 29ms/epoch - 29ms/step
Epoch 3/500000
1/1 - 0s - loss: 0.3212 - mean_absolute_percentage_error: 102.9182 - val_loss: 0.2631 - val_mean_absolute_percentage_error: 100.2923 - 27ms/epoch - 27ms/step
Epoch 4/500000
1/1 - 0s - loss: 0.3117 - mean_absolute_percentage_error: 101.3393 - val_loss: 0.2586 - val_mean_absolute_percentage_error: 100.0617 - 29ms/epoch - 29ms/step
Epoch 5/500000
1/1 - 0s - loss: 0.3065 - mean_absolute_percentage_error: 100.6388 - val_loss: 0.2592 - val_mean_absolute_percentage_error: 100.0828 - 23ms/epoch - 23ms/step
Epoch 6/500000
1/1 - 0s - loss: 0.3048 - mean_absolute_percentage_error: 100.2999 - val_loss: 0.2596 - val_mean_absolute_percentage_e

/home/max/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 - 0s - loss: 0.3037 - mean_absolute_percentage_error: 99.9205 - val_loss: 0.2595 - val_mean_absolute_percentage_error: 100.0000 - 19ms/epoch - 19ms/step
Epoch 10/500000
1/1 - 0s - loss: 0.3037 - mean_absolute_percentage_error: 99.8767 - val_loss: 0.2595 - val_mean_absolute_percentage_error: 100.0000 - 19ms/epoch - 19ms/step
Epoch 11/500000
1/1 - 0s - loss: 0.3037 - mean_absolute_percentage_error: 99.8881 - val_loss: 0.2595 - val_mean_absolute_percentage_error: 100.0000 - 17ms/epoch - 17ms/step
Epoch 12/500000
1/1 - 0s - loss: 0.3036 - mean_absolute_percentage_error: 99.9248 - val_loss: 0.2595 - val_mean_absolute_percentage_error: 100.0000 - 19ms/epoch - 19ms/step
Epoch 13/500000
1/1 - 0s - loss: 0.3036 - mean_absolute_percentage_error: 99.9556 - val_loss: 0.2595 - val_mean_absolute_percentage_error: 100.0000 - 19ms/epoch - 19ms/step
Epoch 14/500000
1/1 - 0s - loss: 0.3036 - mean_absolute_percentage_error: 99.9760 - val_loss: 0.2595 - val_mean_absolute_percentage_error: 100.0000 - 1

In [300]:
# Evaluate the model
loss, mae = model.evaluate(X_train, y_train)
print("Train Loss:", loss, "Train MAE:", mae)

loss, mae = model.evaluate(X_test, y_test)
print("Test Loss:", loss, "Test MAE:", mae)

31/31 [==============================] - 0s 931us/step - loss: 0.2788 - mean_absolute_percentage_error: 106.5773
Train Loss: 0.2788115441799164 Train MAE: 106.57731628417969
2/2 [==============================] - 0s 2ms/step - loss: 0.2617 - mean_absolute_percentage_error: 106.5957
Test Loss: 0.2616594433784485 Test MAE: 106.59571075439453


In [301]:
# Save the model
model.save('my_model')

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(6, 40), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f595812df60>, 140021708247632), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(6, 40), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f595812df60>, 140021708247632), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(40,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f595812f370>, 140021708247312), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(40,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f595812f370>, 140021708247312), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(40, 40), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f5958140040>, 140021707980544), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(40, 1), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f5958141ed0>, 140021707976144), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f5958142e60>, 140021707975264), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f5958142e60>, 140021707975264), {}).


INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


In [302]:



# Make predictions
y_pre = model.predict(X_test)
y_pre = [i[0] for i in y_pre]

# Calculate and save errors
# error = (y_test - y_pre) / y_test * 100
# error = error.sort_values()
# error.to_csv('error.csv', index=False)

2/2 [==============================] - 0s 1ms/step


In [303]:
y_test

array([[-0.6737538 ],
       [-0.58370556],
       [ 0.47543327],
       [ 0.18670716],
       [-0.32428086],
       [ 0.43183848],
       [ 0.01232803],
       [-0.32285153],
       [ 0.35322494],
       [-0.69269251],
       [-0.39360372],
       [-0.43076648],
       [-0.75951403],
       [-0.37657316],
       [-0.19135251],
       [-0.98139718],
       [ 0.70841522],
       [-0.65374308],
       [ 0.26250491],
       [-0.04091478],
       [-0.00591031],
       [ 0.82538503],
       [-0.17777381],
       [ 0.20100054],
       [-0.48865464],
       [-0.64533857],
       [-0.68876184],
       [-0.87957835],
       [ 0.49615866],
       [ 0.09187064],
       [ 0.81837413],
       [-0.49951045],
       [-0.46649991],
       [ 0.28533143],
       [ 0.85134894],
       [ 0.2746114 ],
       [ 0.37823834],
       [ 0.3267822 ],
       [-0.34214758],
       [-0.77773807],
       [ 0.82219046],
       [-0.34000357],
       [ 0.02590674],
       [-0.65660175],
       [-0.91531177],
       [-0

In [304]:
y_pre

[0.0,
 0.0,
 0.11321415,
 0.049743634,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.61847126,
 0.046960887,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.8344012,
 0.70061076,
 0.0,
 0.0,
 0.0,
 0.0,
 0.124038346,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3712291,
 0.0,
 0.025117388,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]